# Project Scheduling (LP)
Solve a precedence-constrained project scheduling problem by minimizing makespan with linear programming.

## Inputs

durations.txt: [task_id, duration]  

$d_i$= duration of job i

precedences.txt: [successor, predecessor] 

𝑇 = number of tasks

## Decision variables

$x_𝑖$ ≥ 0: start time of the job i 

## Model

**Objective:** minimize z = $x_𝑇$ + 1

**Subject to:**

precedence constraints (P)

$x_i$ − $x_j$ ≥ $𝑑_j$            

(i, j) ∈ P

 
$x_𝑖$ ≥ 0 

i = 0, 1, 2, . . . , 𝑇, 𝑇 + 1

In [52]:
# load libraries
import numpy as np
import scipy.sparse as sp

import cplex as cp

In [53]:
def linear_programming(direction, A, senses, b, c, l, u):
    # create an empty optimization problem
    prob = cp.Cplex()

    # add decision variables to the problem including their coefficients in objective and ranges
    prob.variables.add(obj = c.tolist(), lb = l.tolist(), ub = u.tolist())

    # define problem type
    if direction == "maximize":
        prob.objective.set_sense(prob.objective.sense.maximize)
    else:
        prob.objective.set_sense(prob.objective.sense.minimize)

    # add constraints to the problem including their directions and right-hand side values
    prob.linear_constraints.add(senses = senses.tolist(), rhs = b.tolist())

    # add coefficients for each constraint
    row_indices, col_indices = A.nonzero()
    prob.linear_constraints.set_coefficients(zip(row_indices.tolist(), col_indices.tolist(), A.data.tolist()))

    # solve the problem
    prob.solve()

    # check the solution status
    print(prob.solution.get_status())
    print(prob.solution.status[prob.solution.get_status()])

    # get the solution
    x_star = prob.solution.get_values()
    obj_star = prob.solution.get_objective_value()

    return(x_star, obj_star)

In [54]:
def project_scheduling_problem(durations_file, precedences_file):
    
    durations   = np.loadtxt(durations_file, dtype=float) 
    precedences = np.loadtxt(precedences_file, dtype=int)
    if precedences.ndim == 1:
        precedences = precedences.reshape(1, 2)

    # Decision variables
    max_id = int(np.max(durations[:, 0]))
    N = max_id + 2

 
    durations = np.vstack(([0, 0.0], durations, [N - 1, 0.0]))

    d = np.zeros(N, dtype=float)
    d[durations[:, 0].astype(int)] = durations[:, 1].astype(float)

  
    tasks = np.arange(1, N - 1, dtype=int)
    P = np.vstack((
        np.column_stack((tasks, np.zeros_like(tasks))),         # s_i - s_0 >= d_0 (=0)  -> s_i >= 0
        precedences.astype(int),                                # s_succ - s_pred >= d_pred
        np.column_stack((np.full_like(tasks, N - 1), tasks))    # s_T - s_i >= d_i       -> makespan
    ))
    M = P.shape[0]

    
    c = np.concatenate((np.zeros(N - 1, dtype=float), [1.0]))

    #lower and upper bound
    l = np.zeros(N, dtype=float)
    u = np.repeat(cp.infinity, N)

    # RHS
    b = d[P[:, 1].astype(int)]

    # Matrix A
    aij = np.repeat([+1.0, -1.0], M)
    row = np.concatenate((np.arange(M), np.arange(M)))
    col = np.concatenate((P[:, 0].astype(int), P[:, 1].astype(int)))
    A = sp.csr_matrix((aij, (row, col)), shape=(M, N))

    senses = np.repeat("G", M)
   
    x_star, obj_star = linear_programming("minimize", A, senses, b, c, l, u)
    return(x_star, obj_star)

In [55]:
x_star, obj_star = project_scheduling_problem("durations.txt", "precedences.txt")
print(x_star)
print(obj_star)

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 26 rows and 10 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)
1
optimal
[0.0, 0.0, 2.0, 2.0, 5.0, 5.0, 7.5, 9.5, 10.5, 13.5]
13.5
